In [1]:
# your code here

In [125]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import requests
import re 
from bs4 import BeautifulSoup as bs

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r
response = requests.get('https://the-one-api.herokuapp.com/v1/quote', auth=BearerAuth('l70E7VrdrFk2T1VvDoeZ'))
response
results=response.json()
results
data_quote=pd.DataFrame(response.json()['docs'])
data_quote
##This api shows the quote and the 









,_id,dialog,movie,character
0,5cd96e05de30eff6ebcce7e9,Deagol!,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfe9e
1,5cd96e05de30eff6ebcce7ea,Deagol!,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfe9e
2,5cd96e05de30eff6ebcce7eb,Deagol!,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfe9e
3,5cd96e05de30eff6ebcce7ec,Give us that! Deagol my love,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfe9e
4,5cd96e05de30eff6ebcce7ed,Why?,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfca7
...,...,...,...,...
2385,5cd96e05de30eff6ebccf13c,Merry!,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfe2e
2386,5cd96e05de30eff6ebccf13d,Merry!,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfbe6
2387,5cd96e05de30eff6ebccf13e,He's always followed me everywhere I went sinc...,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfc7c
2388,5cd96e05de30eff6ebccf13f,One thing I've learnt about Hobbits: They are ...,5cd95395de30eff6ebccde5d,5cd99d4bde30eff6ebccfbe6


In [55]:
scripts_=pd.read_csv('/Users/nayibahued/Desktop/web-project/scripts.csv')

In [58]:

cols = []
cols = scripts_.columns
cols = [i.upper() for i in cols]
scripts_.columns = cols
scripts_
scripts_=scripts_.rename(columns={'MOVIE':'NAME'})
scripts_


,UNNAMED: 0,CHAR,DIALOG,NAME
0,0,DEAGOL,"Oh Smeagol Ive got one! , Ive got a fish Smeag...",The Return of the King
1,1,SMEAGOL,"Pull it in! Go on, go on, go on, pull it in!",The Return of the King
2,2,DEAGOL,Arrghh!,The Return of the King
3,3,SMEAGOL,Deagol!,The Return of the King
4,4,SMEAGOL,Deagol!,The Return of the King
...,...,...,...,...
2385,2385,PIPPIN,Merry!,The Return of the King
2386,2386,ARAGORN,Merry!,The Return of the King
2387,2387,MERRY,He's always followed me everywhere I went sinc...,The Return of the King
2388,2388,ARAGORN,One thing I've learnt about Hobbits: They are ...,The Return of the King


In [94]:
response_mov = requests.get('https://the-one-api.herokuapp.com/v1/movie', auth=BearerAuth('l70E7VrdrFk2T1VvDoeZ'))

results_mov=response_mov.json()
results_mov
movies_=pd.DataFrame(response_mov.json()['docs'])
movies_
##Im gonna drop the 'series' columns because they represent the joint of the 3 LOTR and The Hobbit movies
##And the hobbits series too
movies_.drop(movies_.index[[0,1,2,3,4]], inplace=True)
movies_
movies_.drop(['budgetInMillions','boxOfficeRevenueInMillions'], axis=1, inplace=True)
movies_
##Im appending the year of release in a new column
Release_year=[2002,2001,2003]
movies_['Release_year']=Release_year
movies_
data_mov= movies_.sort_values(by=['Release_year'], ascending=True)
#data_mov.rename({'name':'movie'}, axis=1)
data_mov
cols_mov = []
cols_mov = data_mov.columns
cols_mov = [i.upper() for i in cols_mov]
data_mov.columns = cols_mov
data_mov

,_ID,NAME,RUNTIMEINMINUTES,ACADEMYAWARDNOMINATIONS,ACADEMYAWARDWINS,RELEASE_YEAR
6,5cd95395de30eff6ebccde5c,The Fellowship of the Ring,178,13,4,2001
5,5cd95395de30eff6ebccde5b,The Two Towers,179,6,2,2002
7,5cd95395de30eff6ebccde5d,The Return of the King,201,11,11,2003


In [123]:
##Merging the csv with the api, to add movie info
quote_movie_info=pd.merge(scripts_,data_mov,on='NAME',how='inner')
quote_movie_info.rename(columns={'UNNAMED: 0':'ORDER'})
df_1=quote_movie_info.copy()
df_1.drop(['UNNAMED: 0'], axis=1, inplace=True)
df_1['DIALOG'].unique()
df_1['DIALOG']=df_1['DIALOG'].str.strip()
df_1['DIALOG']=df_1['DIALOG'].str.replace('\xa0','')
df_1['DIALOG'].unique()
#Now that I cleaned the dialogs, I would like to sort them by the length
dia=list(df_1['DIALOG'])
lgth=[]
dia
for entry in dia:
    l = len(entry)
    lgth.append(l)
    
lgth
#Im gonna add the lgth list to sort the dialogs
df_1['DL_LEN'] = lgth
df_1
df_2 = df_1.sort_values(by=['DL_LEN'], ascending=False)
df_2
    

,CHAR,DIALOG,NAME,_ID,RUNTIMEINMINUTES,ACADEMYAWARDNOMINATIONS,ACADEMYAWARDWINS,RELEASE_YEAR,DL_LEN
552,GANDALF,"Sauron fears you, Aragorn.He fears what you ma...",The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,814
270,DENETHOR,"It's dangerous, I know. Ever the Ring will...",The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,396
153,SAM,"I know. It's all wrong. ,By rights, we sh...",The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,387
791,EOMER,Theoden no longer recognizesfriend from foe. N...,The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,366
155,SAM VOICEOVER,"But in the end, it's only a passing thing........",The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,352
...,...,...,...,...,...,...,...,...,...
896,GIMLI,oh!,The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,3
380,SMEAGOL,No.,The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,3
410,GOLLUM,No.,The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,3
1006,TREEBEARD,,The Two Towers,5cd95395de30eff6ebccde5b,179,6,2,2002,0


In [140]:
#Now for the web-scrape, I´m gonna add the ratings from imdb for each movie

##Return of the king
url = 'https://www.imdb.com/title/tt0167260/?ref_=rvi_tt'
wd = requests.get(url)
wd

wd_cont=wd.content
soup_3=bs(wd_cont, 'html.parser')
score=[]

table_3 = soup_3.find("div",{"class":"title-overview"})
table_4 = soup_3.find("div",{"class":"metacriticScore score_favorable titleReviewBarSubItem"})

for scr in table_3.find_all("strong"):
    score.append(scr.text)

for met in table_4.find_all("span"):
    score.append(met.text)

score






['8.9', '94']

In [154]:
##Fellowship
    
url = 'https://www.imdb.com/title/tt0120737/?ref_=rvi_tt'
wd_1 = requests.get(url)
wd_1

wd1_cont=wd_1.content
soup_4=bs(wd1_cont, 'html.parser')
score_1=[]

table_5 = soup_4.find("div",{"class":"title-overview"})
table_6 = soup_4.find("div",{"class":"metacriticScore score_favorable titleReviewBarSubItem"})

for sr in table_5.find_all("strong"):
    score_1.append(sr.text)

for mt in table_6.find_all("span"):
    score_1.append(mt.text)

score_1



['8.8', '92']

In [159]:
##Two towers
url = 'https://www.imdb.com/title/tt0167261/'
wd_2 = requests.get(url)
wd_2

wd2_cont=wd_2.content
soup_5=bs(wd2_cont, 'html.parser')
score_2=[]

table_7 = soup_5.find("div",{"class":"title-overview"})
table_8 = soup_5.find("div",{"class":"metacriticScore score_favorable titleReviewBarSubItem"})

for sr in table_7.find_all("strong"):
    score_2.append(sr.text)

for mt in table_8.find_all("span"):
    score_2.append(mt.text)

score_2

['8.7', '87']

In [169]:
movs=['The Return of the King','The Fellowship of the Ring','The Two Towers']
final_lst=[]

final_lst.append(score)
final_lst.append(score_1)
final_lst.append(score_2)
final_lst

[['8.9', '94'], ['8.8', '92'], ['8.7', '87']]

In [170]:
web_scr = pd.DataFrame(columns=['IMDB','METASCORE'], data=final_lst)

In [171]:
web_scr

,IMDB,METASCORE
0,8.9,94
1,8.8,92
2,8.7,87


In [173]:
web_scr['NAME']=movs
web_scr

,IMDB,METASCORE,NAME
0,8.9,94,The Return of the King
1,8.8,92,The Fellowship of the Ring
2,8.7,87,The Two Towers


In [184]:
df_3=pd.merge(df_2,web_scr,on='NAME',how='outer')

In [186]:
df_2.to_csv('/Users/nayibahued/Desktop/web-project/lotr.csv', index = False)
